# Update Model Parameters API Examples

This demonstrates how to use the [update_model_parameters](https://siliconlabs.github.io/mltk/docs/python_api/operations/update_model_parameters.html) API.

Refer to the [Model Parameters](https://siliconlabs.github.io/mltk/docs/guides/model_parameters.html) guide for more details.

__NOTES:__  

- Click here: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/siliconlabs/mltk/blob/master/mltk/examples/update_params.ipynb) to run this example interactively in your browser  
- Refer to the [Notebook Examples Guide](https://siliconlabs.github.io/mltk/docs/guides/notebook_examples_guide.html) for how to run this example locally in VSCode  

## Install MLTK Python Package

In [1]:
# Install the MLTK Python package (if necessary)
!pip install --upgrade silabs-mltk

## Import Python Packages

In [1]:
# Import the necessary MLTK APIs
from mltk.core import update_model_parameters, summarize_model

## Example 1: Update model specification

The most common use case of the [update_model_parameters](https://siliconlabs.github.io/mltk/docs/python_api/operations/update_model_parameters.html) API is:  
1. Fully [train](https://siliconlabs.github.io/mltk/docs/guides/model_training.html) a model  
2. Later modify the [model specification](https://siliconlabs.github.io/mltk/docs/guides/model_specification.html) script with additional parameters  
3. Run the [update_model_parameters](https://siliconlabs.github.io/mltk/docs/python_api/operations/update_model_parameters.html) API to update the `.tflite` model file in the [model archive](https://siliconlabs.github.io/mltk/docs/guides/model_archive.html).

In this example, it's assumed that the [MltkModel.model_parameters](https://siliconlabs.github.io/mltk/docs/python_api/mltk_model/model.html#mltk.core.MltkModel.model_parameters) settings in
the [tflite_micro_speech](https://siliconlabs.github.io/mltk/docs/python_api/models/tflite_micro/tflite_micro_speech.html) model specification script have been modified _after_ the model have been
trained.

```python
my_model.model_parameters['average_window_duration_ms'] = 1000
my_model.model_parameters['detection_threshold'] = 185
my_model.model_parameters['suppression_ms'] = 1500
my_model.model_parameters['minimum_count'] = 3
my_model.model_parameters['volume_db'] = 5.0
my_model.model_parameters['latency_ms'] = 0
my_model.model_parameters['log_level'] = 'info'
```

After this API completes, the `tflite_micro_speech.mltk.zip` model archive is updated with a new `tflite_micro_speech.tflite` model file.  
Note that _only_ the parameters in the `.tflite`'s metadata section are modified. The model weights and layers are untouched.

In [2]:
# Update the model parameters
update_model_parameters('tflite_micro_speech')

# Generate a summary of the updated model with new parameters
print(summarize_model('tflite_micro_speech'))

Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 9600
Updating c:/users/reed/workspace/silabs/mltk/mltk/models/tflite_micro/tflite_micro_speech.mltk.zip
Model: "tflite_micro_speech"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 20, 8)         648       
                                                                 
 batch_normalization (BatchN  (None, 25, 20, 8)        32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 25, 20, 8)         0         
                                                                 
 dropout (Dropout)           (None, 25, 20, 8)         0         
                                                          

## Example 2: Update with argument

The [update_model_parameters](https://siliconlabs.github.io/mltk/docs/python_api/operations/update_model_parameters.html) API also works with `.tflite` model files generated outside of the MLTK.  
In this mode, model parameters can be supplied via API argument, e.g.:

In [3]:
import os 
import tempfile
import urllib
import shutil

# Use .tflite mode found here:
# https://github.com/mlcommons/tiny/tree/master/benchmark/training/keyword_spotting/trained_models
# NOTE: Update this URL to point to your model if necessary
TFLITE_MODEL_URL = 'https://github.com/mlcommons/tiny/raw/master/benchmark/training/keyword_spotting/trained_models/kws_ref_model.tflite'

# Download the .tflite file and save to the temp dir
external_tflite_path = os.path.normpath(f'{tempfile.gettempdir()}/kws_ref_model.tflite')
with open(external_tflite_path, 'wb') as dst:
    with urllib.request.urlopen(TFLITE_MODEL_URL) as src:
        shutil.copyfileobj(src, dst)

In [4]:
# Set the parameters in a Python dictionary
parameters = {
    "volume": 10.0,
    "log_level": "debug",
    "threshold": 43
}
# Update the model parameters
update_model_parameters(external_tflite_path, params=parameters)

# Generate a summary of the updated model with new parameters
print(summarize_model(external_tflite_path))

Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 25472
+-------+-------------------+----------------+----------------+-------------------------------------------------------+
| Index | OpCode            | Input(s)       | Output(s)      | Config                                                |
+-------+-------------------+----------------+----------------+-------------------------------------------------------+
| 0     | conv_2d           | 49x10x1 (int8) | 25x5x64 (int8) | Padding:same stride:2x2 activation:relu               |
|       |                   | 10x4x1 (int8)  |                |                                                       |
|       |                   | 64 (int32)     |                |                                                       |
| 1     | depthwise_conv_2d | 25x5x64 (int8) | 25x5x64 (int8) | Multiplier:1 padding:same stride:1x1 activation:relu  |
|    

## Example 3: Return TfliteModel instance

Rather than update the given model, the [update_model_parameters](https://siliconlabs.github.io/mltk/docs/python_api/operations/update_model_parameters.html) API can also return a [TfliteModel](https://siliconlabs.github.io/mltk/docs/python_api/tflite_model/index.html) instance with the updated parameters.

This is done by specifying the `output='tflite_model'` API argument:

In [5]:
# Set the parameters in a Python dictionary
parameters = {
    "volume": 10.0,
    "log_level": "debug",
    "threshold": 43
}
# Generate a TfliteModel instance with the given parameters
# NOTE: The input external_tflite_path file is NOT modified
tflite_model = update_model_parameters(external_tflite_path, params=parameters, output='tflite_model')

# Generate a summary of the returned TfliteModel instance
print(summarize_model(tflite_model))

Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 25472
+-------+-------------------+----------------+----------------+-------------------------------------------------------+
| Index | OpCode            | Input(s)       | Output(s)      | Config                                                |
+-------+-------------------+----------------+----------------+-------------------------------------------------------+
| 0     | conv_2d           | 49x10x1 (int8) | 25x5x64 (int8) | Padding:same stride:2x2 activation:relu               |
|       |                   | 10x4x1 (int8)  |                |                                                       |
|       |                   | 64 (int32)     |                |                                                       |
| 1     | depthwise_conv_2d | 25x5x64 (int8) | 25x5x64 (int8) | Multiplier:1 padding:same stride:1x1 activation:relu  |
|    